In [1]:
from collections import namedtuple
import math

import numpy as np
from ortools.linear_solver import pywraplp

In [2]:
Point = namedtuple("Point", ['x', 'y'])
Facility = namedtuple("Facility", ['index', 'setup_cost', 'capacity', 'location'])
Customer = namedtuple("Customer", ['index', 'demand', 'location'])

def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [3]:
with open("data/fl_50_1", 'r') as input_data_file:
    input_data = input_data_file.read()

lines = input_data.split('\n')

parts = lines[0].split()
facility_count = int(parts[0])
customer_count = int(parts[1])

facilities = []
for i in range(1, facility_count+1):
    parts = lines[i].split()
    facilities.append(Facility(i-1, float(parts[0]), int(parts[1]), Point(float(parts[2]), float(parts[3])) ))

customers = []
for i in range(facility_count+1, facility_count+1+customer_count):
    parts = lines[i].split()
    customers.append(Customer(i-1-facility_count, int(parts[0]), Point(float(parts[1]), float(parts[2]))))


In [4]:
def calculate_distance_matrix(customers, facilities):

    dist_matrix = np.zeros([len(customers), len(facilities)])

    for x, i in enumerate(customers):
        for y, j in enumerate(facilities):

            distance = length(i.location, j.location)

            dist_matrix[x][y] = distance

    return dist_matrix

In [5]:
print(facility_count)
print(customer_count)

50
50


In [6]:
dist_matrix = calculate_distance_matrix(customers, facilities)

In [7]:
solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [8]:
x = []
for i in range(len(customers)):
    t = []
    for j in range(len(facilities)):
        t.append(solver.BoolVar(f'x{i},{j}'))

    x.append(t)

y = [solver.BoolVar(f'y{j}') for j in range(len(facilities))]


In [9]:
for i in range(len(customers)):
    solver.Add(sum([x[i][j] for j in range(len(facilities))]) == customers[i].demand)    

# Constraint: a customer must be assigned to an open facility.
for i in range(len(customers)):
    for j in range(len(facilities)):
        solver.Add(x[i][j] <= (y[j]*customers[i].demand))

# Constraint: the capacity of each facility must not be exceeded.
for j in range(len(facilities)):
    solver.Add(sum([x[i][j] for i in range(len(customers))]) <= (facilities[j].capacity * y[j]))


In [10]:
solver.Minimize(solver.Sum([y[j] * facilities[j].setup_cost for j in range(len(facilities))]) +
                solver.Sum([dist_matrix[i][j] * x[i][j] for i in range(len(customers))
                                                        for j in range(len(facilities))]))

In [11]:
SEC = 1000
MIN = 60 * SEC
solver.SetTimeLimit(1 * MIN)

result_status = solver.Solve()

In [12]:
total_cost = solver.Objective().Value()
solution = [j for i in range(len(customers))
              for j in range(len(facilities))
               if x[i][j].solution_value() > 0]

In [13]:
print(total_cost)
print(solution)

0.0
[]


## pyscipopt

In [14]:
from pyscipopt import Model, quicksum, multidict

In [15]:
def flp(I,J,d,M,f,c):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == 1, "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] * d[i] for i in I) <= M[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= y[j], "Strong(%s,%s)"%(i,j))
    model.setObjective(
        quicksum(f[j]*y[j] for j in J) +
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [16]:
sci_customers = {1+v.index: v.demand for v in customers}
sci_facilities = {1+v.index: [v.capacity, v.setup_cost] for v in facilities}

cost = {}
for i in range(dist_matrix.shape[0]):
    for j in range(dist_matrix.shape[1]):
        cost[(i+1,j+1)] = dist_matrix[i][j]

In [17]:
 I, d = multidict(sci_customers)
 J, M, f = multidict(sci_facilities)

In [18]:
model = flp(I, J, d, M, f, cost)
model.optimize()
EPS = 1.e-6
x,y = model.data

In [19]:
total_cost = model.getObjVal()
solution = [j-1 for i in sci_customers.keys() for j in sci_facilities.keys() if model.getVal(x[i,j]) > EPS]

In [20]:
print(total_cost)
print(solution)

2825999.219090653
[1, 1, 1, 9, 1, 1, 14, 1, 1, 1, 22, 1, 9, 1, 1, 1, 1, 14, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 33, 1, 1, 49, 9, 9, 1, 9, 9, 1, 1, 9, 9, 1, 1, 1, 1]
